In [ ]:
import warnings
warnings.filterwarnings("ignore")
import sys
import cv2
from tracker import Tracker 
from utils import select_bounding_box
from video_read_thread import FileVideoStream
import time
import imutils
from random import randint
from imutils.video import FPS
from people_recognize import PersonRecognizer
import numpy as np
from PIL import Image

In [ ]:
detection_model_path = 'haarcascade_files/haarcascade_frontalface_default.xml'
face_detection = cv2.CascadeClassifier(detection_model_path)
person_detector = PersonRecognizer()

def get_faces_from_haar(frame):
    bboxes=[]
    colors=[]
    frame=imutils.resize(frame,width=500)
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    colors.append((randint(0, 255), randint(0, 255), randint(0, 255)))
    faces=face_detection.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(5,5),flags=cv2.CASCADE_SCALE_IMAGE)
    print(len(faces)," faces found in the frame")
    if len(faces)>0:
        for face in faces:
            (x,y,w,h) = faces[0]
            bboxes.append((x,y,w,h))
            frame2=frame.copy()
            frame2 = cv2.rectangle(frame2,(x,y),(x+w,y+h),(255,0,0),2)
            while cv2.waitKey(1) & 0xFF != ord('q'):
                cv2.imshow("Press q to continue",frame2)
    return bboxes,colors

In [ ]:
def detect_people_from_frame(frame,recognizer=person_detector):
    height,width,channels=frame.shape
    boxes = recognizer.detect_people(frame)
    bboxes =[]
    colors = []
    for box in boxes :
        bboxes.append((int(box[-4]*width/512),int(box[-3]*height/512),int(box[-2]*width/512),int(box[-1])*height/512))
        colors.append((randint(0, 255), randint(0, 255), randint(0, 255)))
    return bboxes,colors

In [ ]:
# Set video to load
videoPath = r"video/VID20200315124213.mp4"
#videoPath = r"C:\Users\HP\PYT\Repos_dowloaded\people-counting-opencv\videos/example_01.mp4"
# Create a video capture object to read videos
cap=FileVideoStream(videoPath,queueSize=64).start() 
# Read first frame
success, frame = cap.read()
# quit if unable to read the video file
#time.sleep(1.0)
#if not success:
#    print('Failed to read video')
#    sys.exit(1)


In [ ]:
FRAMES_PER_DETECT = 30

In [ ]:
detect_times = []
update_times = []

In [ ]:
#print(bboxes)
#bboxes,colors=select_bounding_box(frame)
#bboxes,colors=get_faces_from_haar(frame)

fps = FPS().start()
fr = FRAMES_PER_DETECT
bboxes,colors = detect_people_from_frame(frame)
multiTracker = Tracker(bboxes,frame,trackerType="MOSSE").multiTracker
# Process video and track objects
#fourcc = cv2.VideoWriter_fourcc(*"XVID")
#writer = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30,(720,1280), True)
frames = []
frame_no = 0
#print("Writer created :",writer.isOpened())
while cap.more():
    success, frame = cap.read()
    fr -=1 
    if fr == 0 :
        fr = FRAMES_PER_DETECT
    #if not success:
    #    print(" Stopped at :")
    #    break 
    frame_no += 1
    if fr == FRAMES_PER_DETECT :
        start_time = time.time()
        bboxes,colors=detect_people_from_frame(frame)
        detect_times.append(time.time()-start_time)
        multiTracker = Tracker(bboxes,frame,trackerType="MOSSE").multiTracker
    # get updated location of objects in subsequent frames
    start_time = time.time()
    success, boxes = multiTracker.update(frame)
    update_times.append(time.time()-start_time)
    # draw tracked objects
    for i, newbox in enumerate(boxes):
        p1 = (int(newbox[0]), int(newbox[1]))
        #p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
        p2 = (int( newbox[2]), int( newbox[3]))
        cv2.rectangle(frame, p1, p2, colors[i], 2, 1)
    # show frame
    cv2.putText(frame,str(cap.qsize()),(30,30),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),2)
    cv2.imshow('MultiTracker', frame)
    frames.append(frame)
        #print(type(frame))
    #writer.write(frame)
    # quit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):  
        print("Waitkey")
        break
    fps.update()
    if cap.more() is False :
        print(" cap.more() is False at Frame",frame_no)
cap.stop()
cv2.destroyAllWindows()
#writer.release()

fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

In [ ]:
resized_frames = []
for frame in frames :
    img = Image.fromarray(frame)
    img = img.resize((640,480))
    resized_frames.append(np.asarray(img))

In [ ]:
writer = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30,(640,480),True)
print(writer.isOpened())
fr = 0
for frame in resized_frames:
    fr += 1
#    writer.write(np.random.randint(0, 255, (480,640,3)).astype('uint8'))
    writer.write(frame.astype('uint8'))
    #print(fr," ",np.mean(frame))
writer.release()

In [ ]:
np.mean(detect_times)

In [ ]:
np.mean(update_times)

In [ ]:
type(np.random.randint(0, 255, (480,640,3)).astype('uint8'))

In [ ]:
np.random.randint(0, 255, (480,640,3)).astype('uint8').shape

In [ ]:
frame.shape

In [ ]:
frame_no

In [ ]:
play_video(r"video/VID20200315124213.mp4")

In [ ]:
def play_video(path):
    vs = FileVideoStream(path).start()
    time.sleep(2)
    frame_no = 0
    while vs.more():
        #print(frame_no,end=" ")
        #if not vs.stream.isOpened():
        #    break
        ret,frame = vs.read()
        #if not ret :
        #    break
        frame_no += 1
        cv2.imshow("Press Q to exit",frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):  
            #print("Waitkey")
            break
    vs.stop()
    cv2.destroyAllWindows()
    return frame_no

In [ ]:
import dlib

In [ ]:
dlib.correlation_tracker()